In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table as Table
from astropy.io import ascii
import matplotlib.pyplot as plt
from linetools.spectra import utils as ltsu
from linetools.spectra.xspectrum1d import XSpectrum1D
import astropy.units as u
from astropy import constants as const
from numpy import random as ran
import tau_eff
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
stack1 = fits.open("/home/jsm/PycharmProjects/tau_eff/fits/composites/stack_1.fits")

model1 = fits.open("/home/jsm/PycharmProjects/tau_eff/SB99/Fitting/data/thesis/model_1.fits")

stack2 = fits.open("/home/jsm/PycharmProjects/tau_eff/fits/composites/stack_2.fits")

model2 = fits.open("/home/jsm/PycharmProjects/tau_eff/SB99/Fitting/data/thesis/model_2.fits")

In [6]:
z1,t1,t1_sig = tau_eff.lya_opacity(stack1,model1,"tau_1.fits")

In [7]:
z2,t2,t2_sig = tau_eff.lya_opacity(stack2,model2,"tau_2.fits")

In [9]:
tau1_table = Table([z1,t1,t1_sig], names = ('redshift','tau','tau_sig'))

tau2_table = Table([z2,t2,t2_sig], names = ('redshift','tau','tau_sig'))

In [10]:
#ascii.write(tau2_table,format= "latex")

In [11]:
#Faucher-Giguere

def F_G(z):
    
    τ_eff = 0.0018*(1+z)**3.92
    
    return τ_eff

In [12]:
#Becker

def Becker(z):
    
    τ_eff = .751*((1+z)/(1+3.5))**2.9 - .132
    
    return τ_eff

In [13]:
#Kirkman 
def D_a_to_τ_eff(D_a):
    
    τ_eff = -np.log(1-D_a)
    
    return τ_eff

z_vals_k = [1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2]

D_a_k = np.array([0.098,0.099,0.128,0.143,0.202,0.213,0.250,0.252,0.338])

D_a_k_sig = np.array([0.015,0.006,0.006,0.013,0.014,0.014,0.016,0.015,0.024])
                 
τ_eff_k = D_a_to_τ_eff(D_a_k)

τ_err_k =  D_a_to_τ_eff(D_a_k_sig)

In [14]:
#Schaye et al

z_vals_s = np.array([1.756,2.000,1.799,2.034,1.843,2.080,1.986,2.224,2.003,2.239,1.998,
            2.242,2.010,2.256,2.103,2.366,2.217,2.496,2.243,2.506,2.308,2.572,
            2.447,2.715,2.509,2.805,2.626,2.920,2.670,3.009,2.752,3.078,3.058,
            3.382,3.088,3.411,3.708,3.912,3.517,3.862,3.862,4.287])


τ_eff_s = np.array([0.099,0.093,0.120,0.162,0.224,0.124,0.128,0.164,0.180,0.113,0.139,
           0.149,0.115,0.156,0.131,0.175,0.137,0.214,0.180,0.205,0.234,0.283,
           0.177,0.308,0.182,0.273,0.279,0.343,0.232,0.360,0.329,0.271,0.423,
           0.496,0.366,0.445,0.705,0.811,0.644,0.843,0.839,0.827])


τ_err_s = np.array([0.016,0.016,0.026,0.026,0.033,0.019,0.019,0.026,0.027,0.018,0.021,
           0.018,0.014,0.021,0.019,0.022,0.020,0.029,0.025,0.022,0.029,0.032,
           0.025,0.041,0.021,0.030,0.033,0.037,0.023,0.040,0.043,0.030,0.034,
           0.048,0.039,0.041,0.074,0.065,0.056,0.075,0.069,0.060])

In [15]:
#thomas 

def Thomas(z):
    
    return .0089*(1+z)**2.55

In [16]:
def lbg(z):
    
    return 0.165*( (1+z)/(1+2.23) )**3.32

In [17]:
# range
zrange = np.linspace(0,4,100)

In [61]:
plt.figure(figsize=(10,5))

#my plot
#plt.errorbar(z1, t1, yerr=t1_sig ,fmt="o",markersize=5, 
#             capsize=5, label="This work ($z_{low}$ & $z_{high}$)",color="#fd8d3c",ecolor="grey")

#plt.errorbar(z2, t2, yerr=t2_sig,fmt="o",markersize=5, 
#             capsize=5, color="#fd8d3c",ecolor="grey")

plt.plot(zrange,lbg(zrange), label="This work", color = "#fd8d3c")

#the others
plt.plot(zrange,Thomas(zrange), label = "Thomas et al. 2017",color="#b10026",ls="-.")
plt.plot(zrange,F_G(zrange), label = "Faucher-Giguere et al. 2008",color="#02818a",ls="dotted")
plt.errorbar(z_vals_k,τ_eff_k, yerr = τ_err_k, capsize=5, 
             marker="*",label="Kirkman et al. 2005",ls="none",color="#f03b20", markersize=10)
plt.plot(zrange,Becker(zrange), label="Becker et al. 2013", color="magenta",ls="--")

#zero line
plt.hlines(0,0,5, color="lightgrey",linestyle="--")

plt.xlim(1.5,3.4)
plt.ylim(-.1,.6)

#labels
plt.xlabel("$z$",fontsize=15)
plt.ylabel("$τ_{eff}$",fontsize=15)

#misc
plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
k_z = fits.Column(name='redshift', array=z_vals_k, format="E")

k_tau = fits.Column(name='tau', array=τ_eff_k, format="E")

k_sig = fits.Column(name='noise', array=τ_err_k, format="E")

k_t = fits.BinTableHDU.from_columns([k_z, k_tau, k_sig])

k_t.writeto("/home/jsm/PycharmProjects/tau_eff/fits/Tau_eff/Kirkman_tau.fits")

In [ ]:
s_z = fits.Column(name='redshift', array=z_vals_s, format="E")

s_tau = fits.Column(name='tau', array=τ_eff_s, format="E")

s_sig = fits.Column(name='noise', array=τ_err_s, format="E")

s_t = fits.BinTableHDU.from_columns([s_z, s_tau, s_sig])

s_t.writeto("/home/jsm/PycharmProjects/tau_eff/fits/Tau_eff/Schaye_tau.fits")